# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from config import *

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = load("Tongyi")


In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:

chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set',
 'text': 'A suitable name for a company that makes Queen Size Sheet Sets could be "QueenThread Bed Linens" or "Regal Sheets Co." These names emphasize the size and the luxurious aspect of the product, appealing to customers who are looking for high-quality bedding options. Other creative possibilities could be "Elegant Queen Collection," "Serenity Sheets," "Palatial Panache," or "Comforting Charm quilts & linens." When choosing a name, consider incorporating elements that reflect comfort, quality, and the target audience\'s preferences.'}

## SimpleSequentialChain

In [19]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = load("Tongyi")

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [17]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [21]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [22]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
A suitable name for a company that makes Queen Size Sheet Sets could be "Regal Linens," "Elegant Sheets," "Serenity Bedding," "Queenly Comfort," "Bedside Boudoir," "Luxury Lavender Linens," "Cloud9 Sheets," or "Soothing Sleep Co." The name should evoke a sense of quality, comfort, and style, reflecting the appeal of the products it offers. Choose a name that resonates well with your target audience and aligns with the image you want to project.
"Serene Majesty: Exquisite Queen Size Sheet Sets for Effortless Luxury and Restful Nights."

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': '"Serene Majesty: Exquisite Queen Size Sheet Sets for Effortless Luxury and Restful Nights."'}

## SequentialChain

In [28]:
from langchain.chains import SequentialChain

In [23]:
llm = load("Tongyi")

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [24]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [25]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [26]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [29]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [31]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, which is strange. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit! ?",
 'summary': "The reviewer believes the product's taste is underwhelming due to poor foam stability, suggesting a possible issue with the batch or authenticity of the product compared to store-bought versions.",
 'followup_message': 'Réponse : Le commentateur exprime sa déception quant au goût du produit, qualifiant son arôme de médiocre et signalant un manque de stabilité à la mousse. Il fait remarquer une différence négative par rapport aux versions achetées en magasin, ce qui laisse entrevoir la possibilité qu\'il s\'agisse d\'un lot périmé ou d\'une imitation. La question posée, "Vieux lot ou contrefaçon!?", met en avant l\'inqu

## Router Chain

In [32]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [33]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [34]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:
llm = load("Tongyi")

In [36]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [37]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [38]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [39]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [40]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [42]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the explanation for black body radiation in terms of quantum mechanics?'}
> Finished chain.


{'input': 'What is the explanation for black body radiation in terms of quantum mechanics?',
 'text': "Blackbody radiation, a concept in classical physics, refers to the thermal electromagnetic radiation emitted by a perfect absorber and emitter of heat, when it's heated to a temperature above absolute zero. The explanation using quantum mechanics, particularly the groundbreaking work of Max Planck, Albert Einstein, and Niels Bohr, brought a revolutionary understanding.\n\nIn quantum mechanics, light and other forms of energy are no longer considered continuous waves but rather consist of discrete packets called quanta. This idea is known as photons for light. Planck introduced the concept of quantized energy levels, where he proposed that the energy of a photon is proportional to its frequency, given by the equation:\n\nE = hν\n\nwhere E is the energy of a photon, h is Planck's constant (approximately 6.626 x 10^-34 Js), and ν is the frequency of the photon.\n\nWhen a blackbody absorb

In [44]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the sum of 2 and 2?'}
> Finished chain.


{'input': 'What is the sum of 2 and 2?',
 'text': 'The sum of 2 and 2 is 4. This is a basic arithmetic operation that involves adding the two numbers together. The result is straightforward and does not require breaking down into component parts.'}

In [45]:
chain.invoke("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA? (biology)'}

ValueError: Received invalid destination chain name 'biology'